# Creating oxidation states lists

We have data-mined a list of oxidation states based on the structures reported in the Inorganic Crystal Structure Database (as of Sep 2024). This tutorial demonstrates how SMACT can be used to filter this pre-compiled list to produce lists of oxidation states that can be used in SMACT workflows.

## 1. Getting started



[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WMD-group/SMACT/blob/master/docs/tutorials/filtering_icsd_oxidation_states.ipynb)

In [1]:
# Install the required packages
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install "smact[strict] @ git+https://github.com/WMD-group/SMACT.git@master" --quiet

!pip install pandarallel "matplotlib-venn[shapely]" --quiet

In [24]:
from smact.utils.oxidation import ICSD24OxStatesFilter
from smact.structure_prediction.utilities import parse_spec
import seaborn as sns
import matplotlib.pyplot as plt
import dash
import plotly.graph_objects as go
import plotly.io as pio
from dash.dependencies import Input, Output
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pymatviz as pmv
from matplotlib_venn import venn3
import smact
from smact.utils.composition import comp_maker
from smact.screening import smact_filter, smact_validity
from mp_api.client import MPRester
import pandas as pd
from pandarallel import pandarallel
from pprint import pprint
import os

# Interactive plotly figures don't show up on GitHub.
# https://github.com/plotly/plotly.py/issues/931
# change renderer from "png" to "notebook" to get hover tooltips back
# (but blank plots on GitHub!)
pio.renderers.default = "png"
sns.set_theme(context="notebook", style="ticks", palette="colorblind")

## 2. Using the oxidation states filter

In [ ]:
# Initialise the oxidation state filter
ox_filter = ICSD24OxStatesFilter()
ox_states_df = ox_filter.ox_states_df
ox_states_df.head(15)

The above code output presents a Dataframe of the occurrences of elements in particular oxidation states mined from the ICSD in September 2024. Here the results count refers to the number of structures in the ICSD that contain the element in a given oxidation state. The mininig process was quite simple, as such there are many 0 values in the table. We can use the built-in method `get_species_occurrences_df` to return a dataframe of the elements, the ionic species reported for a given element, the number of occurrences of a species in the ICSD and the species' proportion with respect to all ionic species of a given element.

In [ ]:
# Return the dataframe with non-zero results
ox_filter.get_species_occurrences_df(sort_by_occurrences=False)

In [ ]:
# We can also return the dataframe sorted by occurrences (results_count)
ox_filter.get_species_occurrences_df(sort_by_occurrences=True)

To directly produce a list of oxidation states that can be used in SMACT workflows, we have built in functions which can be used to filter the data. The method `filter` can be used to filter the data based on the number of occurrences of the oxidation state in the ICSD. The function takes the following arguments:

- `consensus`: Minimum number of occurrences in literature for an ion to be considered valid. It helps to avoid unphysical species which somehow found their way into ICSD (e.g. H<sup>3+</sup>). Default is 3.
- `include_zero`: Include oxidation state of zero in the filtered list. Default is False.
- `commonality`: Excludes species below a certain proportion of appearances in literature with respect to the total number of reports of a given element (after the consensus threshold has been applied). "Low" includes all species, "medium" excludes rare species below 10% occurrence, and "high" excludes non-majority species below 50% occurrence. You can also specify your own threshold (float or int). Default is "low".

In [ ]:
# Return a dataframe with columns element and oxidation state,
# where oxidation state contains the oxidation states of the element in the ICSD which exceed the threshold
ox_filter.filter(consensus=1, include_zero=False, commonality=11)

Using the same set of arguments we can also extract the list of ionic species present above a given threshold.

In [ ]:
species_list_100 = ox_filter.get_species_list(
    consensus=1, include_zero=False, commonality=11
)
species_list_100

### Writing the filtered oxidation states to a file

The filtered oxidation states can be written to a file using the `write` method. This method takes the `consensus`, `include_zero` and `commonality` arguments as well as:

- `filename`: The name of the file to write the filtered oxidation states to.
- `comment`: A comment to write to the file.

In [8]:
# Write the results to a file
ox_filter.write(
    "oxidation_states_icsd24_100.txt",
    consensus=1,
    include_zero=False,
    commonality=11,
    comment="Oxidation states of elements in the 2024 version of the ICSD",
)

In [ ]:
# Read the results from a file
with open("oxidation_states_icsd24_100.txt", "r") as f:
    print(f.read())

## Using custom Oxidation States in SMACT workflows

The oxidation states filter function can be used to create SMACT-compatible oxidation state lists which can be used with the `smact_validity` function for check if a compound passes the SMACT chemical rules or with the `smact_filter` function to enumerate "SMACT-sensible" compositions.

### SMACT Filter
We can enumerate SMACT-sensible compositions using the `smact_filter` function. Here, we will consider the `Zn-Ti-O` system and enumerate all possible compositions of Zn, Ti and O that pass the SMACT chemical rules. 

In [ ]:
# Define the elements for which we want to generate the allowed combinations
elements = ["Zn", "Ti", "O"]

# Create an element dictionary
space = smact.element_dictionary(elements)

# Enumerate the allowed combinations
allowed_combinations_smact14 = smact_filter(
    space.values(), threshold=8, oxidation_states_set="smact14"
)
allowed_combinations_icsd24_100 = smact_filter(
    space.values(), threshold=8, oxidation_states_set="oxidation_states_icsd24_100.txt"
)
allowed_combinations_icsd24 = smact_filter(
    space.values(), threshold=8, oxidation_states_set="icsd24"
)

# Convert to Pymatgen Composition objects
smact14_comps = list(set([comp_maker(c) for c in allowed_combinations_smact14]))
icsd24_100_comps = list(set([comp_maker(c) for c in allowed_combinations_icsd24_100]))
icsd24_comps = list(set([comp_maker(c) for c in allowed_combinations_icsd24]))

# Print the number of allowed combinations
print(
    f"Number of allowed combinations in the SMACT14 set: {len(allowed_combinations_smact14)}"
)
print(
    f"Number of allowed combinations in the ICSD24 (consensus=1, include_zero=False, commonality=11) set: {len(allowed_combinations_icsd24_100)}"
)
print(
    f"Number of allowed combinations in the ICSD24 set: {len(allowed_combinations_icsd24)}"
)

In [ ]:
# Create the unique sets to make venn diagrams

smact14_set = set(smact14_comps)
icsd24_100_set = set(icsd24_100_comps)
icsd24_set = set(icsd24_comps)

# Create the venn diagram

fig, ax = plt.subplots(figsize=(5, 5))

v = venn3(
    (smact14_set, icsd24_100_set, icsd24_set),
    set_labels=("SMACT14", "ICSD24 filtered", "ICSD24"),
)

plt.show()

The Venn diagram provides a visual representation of the number of compositions that pass the SMACT filter using different sets of oxidation states.
As we can see, the number of compositions that pass the SMACT filter changes depending on the oxidation states used in the filter. This suggests that the choice of oxidation states is important when using the SMACT filter. Additionally, some oxidation state lists can result in unique compositions that would not be found using other oxidation state lists.

It is expected that the filtered ICSD24 list would be a subset of the ICSD24 oxidation state list. The argument settings reduce the possible compositions such that all of the enumerated compositions are a subset of the composition space enumerated using the 2014 SMACT oxidation state list.

In [ ]:
# Show the compositions unique to the ICSD24 set
pprint(icsd24_set - smact14_set)

We can see that from the list above, the unique compositions from using the ICSD 24 list all contain Zn<sup>3+</sup> which shows that it is not present in the manually compiled 2014 SMACT oxidation states list. Application of filtering criteria eliminates the compositions conataining the Zn<sup>3+</sup> oxidation state, showing that while present in the ICSD, it is not a common oxidation state for Zn.

In [ ]:
pprint(icsd24_100_set - smact14_set)

### SMACT Validity
We can check if a compound passes the SMACT chemical rules using the `smact_validity` function.

Note: This function is quite simple and as such currently fails for mixed valence compounds. This is something we are working on improving in the future.

In [ ]:
# Query the Materials Project API for all materials that have been experimentally synthesised
# You'll need to input your API key if it's not already set in the environment
with MPRester(use_document_model=False) as mpr:
    docs = mpr.materials.summary.search(
        theoretical=False, fields=["material_id", "formula_pretty", "energy_above_hull"]
    )

# Create a dataframe from the results

mp_df = pd.DataFrame(docs)
mp_df.head(10)

In [ ]:
# Sort the dataframe by formula_pretty and energy_above_hull and drop duplicate formula by keeping the lowest energy above hull
mp_df = (
    mp_df.sort_values(["formula_pretty", "energy_above_hull"])
    .drop_duplicates("formula_pretty")
    .reset_index(drop=True)
)
mp_df.head(10)

In [ ]:
supplied_oxidation_states = [
    "smact14",
    "icsd24",
    "icsd16",
    "pymatgen_sp",
    "oxidation_states_icsd24_100.txt",
]

pandarallel.initialize(progress_bar=True)

In [ ]:
for ox_list in supplied_oxidation_states:
    mp_df[f"{ox_list}"] = mp_df["formula_pretty"].parallel_apply(
        lambda x: smact_validity(x, oxidation_states_set=ox_list)
    )

mp_df.head(10)

In [ ]:
# Plot the number of valid structures for each oxidation state set


valid_counts = mp_df[[f"{ox_list}" for ox_list in supplied_oxidation_states]].sum()
# Append the total number of structures to the valid_counts series
valid_counts = pd.concat([valid_counts, pd.Series([len(mp_df)], index=["Total"])])
# Relabel the index oxidation_states_icsd24_100.txt to icsd24_100
valid_counts.rename(
    index={"oxidation_states_icsd24_100.txt": "icsd24_100"}, inplace=True
)
fig, ax = plt.subplots(figsize=(16, 8))
# Show the numbers on the bars
for i, v in enumerate(valid_counts.values):
    ax.text(i, v + 10, str(v), ha="center", va="bottom")
sns.barplot(x=valid_counts.index, y=valid_counts.values, ax=ax)
ax.set_xlabel("Oxidation state set")
ax.set_ylabel("Number of valid structures")

#
# Include total number of structures in the title

total_structures = len(mp_df)
ax.set_title(
    f"Number of valid structures for each oxidation state set (Total structures: {total_structures})"
)
plt.show()